In [2]:
import pandas as pd
import numpy as np
from pprint import pprint
import re
import os

from src.my_functions import *

In [3]:
original_dataset=pd.read_csv('input/attacks.csv', encoding='latin-1')

At first sight we can see a 23 columns dataset. Some of them are dates, some of them are categorical data and links to files than can be useful if important information is missed.

In [4]:
original_dataset.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


We create a copy of the original dataset for security

In [5]:
cleaned_dataset=original_dataset.copy()

# Looking for NaN values

First of all we are going to check the two last columns that appears to be only NaN values

Since the original data set has 25723 rows and the columns Unnamed:22 and Unnamed:23 have 25722  NaN values we can safely drop them.

In [6]:
sum(cleaned_dataset['Unnamed: 22'].isna())

25722

In [7]:
cleaned_dataset=cleaned_dataset.drop(columns=["Unnamed: 23","Unnamed: 22"], axis=1)

In the cell bellow we can see that we are not almost near to detect all the missing values. Comparing the number of NaN to the number of data, it seems that there are lots of rows that are empty. Let's check that by creating a new column that shows how many NaN values has each row,

In [8]:
cleaned_dataset.isna().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
dtype: int64

In [9]:
cleaned_dataset["Is_all_NaN"]=cleaned_dataset.isna().sum(axis=1)

cleaned_dataset["Is_all_NaN"].value_counts()

22    17020
21     2394
1      1516
0      1422
2      1200
3      1196
4       540
5       293
6       102
7        26
8         7
20        7
Name: Is_all_NaN, dtype: int64

So we can see that there are a great number of columns that have no information or almost no information (columns with 22 or 21). We proceed to remove them. Afther, we will drop the generated "Is_all_NaN"

In [10]:
cleaned_dataset=cleaned_dataset.dropna(axis=0, how='any', thresh=20, subset=None, inplace=False)

In [11]:
cleaned_dataset=cleaned_dataset.drop(columns="Is_all_NaN")

There are still some NaN values, but we will with that later

In [12]:
cleaned_dataset.isna().sum()

Case Number                  0
Date                         0
Year                         0
Type                         1
Country                      2
Area                       103
Location                   144
Activity                   153
Name                        24
Sex                        217
Age                       1929
Injury                       2
Fatal (Y/N)                362
Time                      2422
Species                   2139
Investigator or Source       5
pdf                          0
href formula                 1
href                         0
Case Number.1                0
Case Number.2                0
original order               0
dtype: int64

# Duplicated values

Now we are looking for rows that have duplicated values

In [13]:
sum(cleaned_dataset.duplicated())

0

So there are not rows with duplicated values. Now we look for columns with duplicated values.

In [14]:
cleaned_dataset[['Case Number','Date','Case Number.1','Case Number.2']]

,Case Number,Date,Case Number.1,Case Number.2
0,2018.06.25,25-Jun-2018,2018.06.25,2018.06.25
1,2018.06.18,18-Jun-2018,2018.06.18,2018.06.18
2,2018.06.09,09-Jun-2018,2018.06.09,2018.06.09
3,2018.06.08,08-Jun-2018,2018.06.08,2018.06.08
4,2018.06.04,04-Jun-2018,2018.06.04,2018.06.04
...,...,...,...,...
6290,ND.0012,Before 19-Jul-1913,ND.0012,ND.0012
6296,ND.0006,Before 1906,ND.0006,ND.0006
6297,ND.0005,Before 1903,ND.0005,ND.0005
6299,ND.0003,1900-1905,ND.0003,ND.0003


In [15]:
cleaned_dataset=cleaned_dataset.drop(columns=['Case Number.1','Case Number.2'], axis=1)

So with "Case Number" it was pretty easy to compare by sight, but it is more difficult to compare URLs by sight, so we will do it by checking with boolean logic.

In [16]:
(cleaned_dataset['href formula']==cleaned_dataset['href']).value_counts()

True     5282
False      52
dtype: int64

It seems that both columns are the same but some mistakes had been made. Nonetheless, we cannot drop a column arbitrarily, so lets check the values that are not equal.

In [17]:
pprint(cleaned_dataset[['href','href formula']].loc[cleaned_dataset['href']!=cleaned_dataset['href formula']].head())

                                                  href  \
50   http://sharkattackfile.net/spreadsheets/pdf_di...   
96   http://sharkattackfile.net/spreadsheets/pdf_di...   
131  http://sharkattackfile.net/spreadsheets/pdf_di...   
133  http://sharkattackfile.net/spreadsheets/pdf_di...   
141  http://sharkattackfile.net/spreadsheets/pdf_di...   

                                          href formula  
50   http://sharkattackfile.net/spreadsheets/pdf_di...  
96   http://sharkattackfile.net/spreadsheets/pdf_di...  
131  http://sharkattackfile.net/spreadsheets/pdf_di...  
133  http://sharkattackfile.net/spreadsheets/pdf_di...  
141  http://sharkattackfile.net/spreadsheets/pdf_di...  


When comparing the columns that have different values, we find that none of the URLs are operative anymore. The same is true for the columns that have equal values. This column can be dropped without losing any practical information.

In [18]:
cleaned_dataset=cleaned_dataset.drop(columns=['href','href formula'])

'pdf' is a column listing reports files to which we do not hace access, so there is not usefull information for us there. On the other hand 'original order' is a column of integer numbers (formated as floats) that it is just de order in which the data were stored.

In [19]:
cleaned_dataset=cleaned_dataset.drop(columns=['pdf','original order'])

Now we search for duplicated rows

In [20]:
cleaned_dataset=cleaned_dataset.drop_duplicates()

Now we are finished (at least by the moment) with deleting rows and columns we are going to reset the dataframe indexes in sake of clarity

In [21]:
cleaned_dataset=cleaned_dataset.reset_index(drop=True)

# Consistency of the data 

So now we are going to check whether each column has usefull and well formated information or no. This only can be done in a column by column basis, since no general criteria can be applied. Also, we will check if the datatype is the appropiate for this kind of data.

In [22]:
cleaned_dataset.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source'],
      dtype='object')

In [23]:
cleaned_dataset['Date'].value_counts()

1957                   10
1950                    6
1970s                   5
12-Apr-2001             5
28-Jul-1995             5
                       ..
Reportd 15-Jul-1894     1
4-Apr-1970              1
09-Dec-1964             1
20-Jun-2006             1
08-Mar-1938             1
Name: Date, Length: 4652, dtype: int64

Date column doesn't seem to have well formatted data. There is some uncompleted information about dates, but it still can be usefull in the future. Let's check next columns.

Year column is a more simple column and looks better than date column. However, when we check the minimun of the dataset, we have 0 values. Since we can hardly believe that we have data from year 0, let's try to complete this column with information from "Date" column

In [24]:
cleaned_dataset['Year'].value_counts()

2015.0    139
2017.0    127
2011.0    124
2016.0    123
2014.0    116
         ... 
1703.0      1
1859.0      1
1779.0      1
1748.0      1
1816.0      1
Name: Year, Length: 201, dtype: int64

In [25]:
cleaned_dataset['Date'].loc[cleaned_dataset['Year']==0]

5253         Ca. 336.B.C..
5254          Ca. 725 B.C.
5255          1990 or 1991
5256           Before 2016
5257       Before Oct-2009
               ...        
5329    Before 19-Jul-1913
5330           Before 1906
5331           Before 1903
5332             1900-1905
5333             1845-1853
Name: Date, Length: 81, dtype: object

We see that there is some related information in the column Date when Year is 0, but something comes to the eye. All the data that has 0 in year are acumulated at the end of the dataset(You can see this by looking to the indexes). Seems that those data are added at posteriori and has no consistency with the dataset. We will make further comprobations

There are 5253 values with a value different from 0 grouped in 200 categories (which is a very reasonable number for a period of time) but there are 81 values with a value equal to 0 from a total sample size of 5334

It's clear that those data are not realiable and we can drop it.

In [26]:
cleaned_dataset=cleaned_dataset[cleaned_dataset.Year!=0]

Also Year data need to be casted to integer values

In [27]:
cleaned_dataset['Year']=cleaned_dataset['Year'].astype(int)

In the column 'Type' we can see that values are razonably well categorized except little exceptions that can be easily fixed

In [28]:
cleaned_dataset['Type'].value_counts()

Unprovoked      4001
Provoked         515
Invalid          376
Boating          148
Sea Disaster     107
Boat             102
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [29]:
cleaned_dataset['Type']=cleaned_dataset['Type'].replace('Boat','Boating')
cleaned_dataset['Type']=cleaned_dataset['Type'].replace('Boatomg','Boating')

In [30]:
cleaned_dataset['Type'].value_counts()

Unprovoked      4001
Provoked         515
Invalid          376
Boating          251
Sea Disaster     107
Questionable       2
Name: Type, dtype: int64

'Name' column doesn't seem to have many usefull information. Its five more frequent value are not names, but gender or age, qhich are stored in other columns. Also, particular names are not important for statistical questions we are trying to answer.

In [31]:
cleaned_dataset['Name'].value_counts()

male               423
female              82
boy                 13
2 males             10
child                6
                  ... 
Eva Cameron          1
Crisolog Urizar      1
Lorraine             1
David Lomas          1
Luis Hernandez       1
Name: Name, Length: 4612, dtype: int64

In [32]:
cleaned_dataset=cleaned_dataset.drop(columns='Name')

Sex column is pretty well categorized but some data can be rescued

In [33]:
cleaned_dataset['Sex '].value_counts()

M      4454
F       578
M         2
N         1
lli       1
.         1
Name: Sex , dtype: int64

In [34]:
cleaned_dataset['Sex ']=cleaned_dataset['Sex '].replace('N','F')
cleaned_dataset['Sex ']=cleaned_dataset['Sex '].replace('M ','M')

In [35]:
cleaned_dataset.loc[cleaned_dataset['Sex ']=='lli']

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
1543,2004.11.11.b,11-Nov-2004,2004,Unprovoked,USA,California,"Bunkers, Humboldt Bay, Eureka, Humboldt County",Surfing,lli,38,"Lacerations to hand, knee & thigh",N,13h30,5.5 m [18'] white shark,"R. Collier, GSAF"


In [36]:
cleaned_dataset.loc[cleaned_dataset['Sex ']=='.']

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
4778,1908.06.02.R,Reported 02-Jun-1908,1908,Sea Disaster,PAPUA NEW GUINEA,New Britain,Matupi,.,.,NaN,"Remains of 3 humans recovered from shark, but ...",Y,NaN,Allegedly a 33-foot shark,"Taranaki Herald, 6/2/1908"


In [37]:
cleaned_dataset['Sex ']=cleaned_dataset['Sex '].replace('lli','M')
cleaned_dataset=cleaned_dataset[cleaned_dataset['Sex ']!='.']
cleaned_dataset=cleaned_dataset.rename(columns={'Sex ':'Sex'})

'Age' column have some data values that can be fixed and a quite important cuantity of NaN wrong formatted.

In [38]:
for i in cleaned_dataset.index:
    x=cleaned_dataset.loc[i,'Age']
    list_matches=re.findall(r'\d{1,2}',str(x))
    if len(list_matches)>=1:
        cleaned_dataset.loc[i,'Age']=int(sum(map(int,list_matches))/len(list_matches))
    else:
        cleaned_dataset.loc[i,'Age']=np.nan

'Injury' column has 3262 different values with no coding or standard, so we cannot extract statistical information from here. Anyhow, it will remain because it does have information.

In [39]:
cleaned_dataset['Injury'].value_counts()

FATAL                                                          555
Foot bitten                                                     78
No injury                                                       67
Leg bitten                                                      61
Survived                                                        52
                                                              ... 
FATAL, both arms severed                                         1
Feet lacerated                                                   1
3 toes severed                                                   1
Bumped, then knee bitten                                         1
Laceration to left calf from hooked shark PROVOKED INCIDENT      1
Name: Injury, Length: 3262, dtype: int64

'Fatal (Y/N)' column is mostly well codificated but need some fixes

In [43]:
cleaned_dataset['Fatal (Y/N)'].value_counts()

N          3851
Y          1017
UNKNOWN      22
2017          1
Name: Fatal (Y/N), dtype: int64

In [44]:
cleaned_dataset['Fatal (Y/N)']=cleaned_dataset['Fatal (Y/N)'].replace(' N','N')

In [45]:
cleaned_dataset['Fatal (Y/N)']=cleaned_dataset['Fatal (Y/N)'].replace('M','N')

In [46]:
cleaned_dataset.loc[cleaned_dataset['Fatal (Y/N)']=='2017']

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
743,2012.06.10,10-Jun-2012,2012,Provoked,ITALY,Sardinia,Muravera,Attempting to rescue an injured & beached shark,M,57,Lower left leg injured PROVOKED ACCIDENT,2017,Morning,"Blue shark, 2.5m","D. Puddo, 6/11/2012"


In [47]:
cleaned_dataset['Fatal (Y/N)']=cleaned_dataset['Fatal (Y/N)'].replace('2017','N')
cleaned_dataset=cleaned_dataset.rename(columns={'Fatal (Y/N)':'Fatal'})

'Time' values are not standarized, so we will try to 

In [48]:
cleaned_dataset['Time'].value_counts()

Afternoon                         181
11h00                             127
Morning                           120
15h00                             108
12h00                             108
                                 ... 
19h00 / 20h00                       1
Before daybreak                     1
14h30 / 15h30                       1
Sometime between 06h00 & 08hoo      1
15h56                               1
Name: Time, Length: 360, dtype: int64

In [49]:
cleaned_dataset.Time=cleaned_dataset.Time.apply(assign_time)

'Species' column is not pretended to be used in further analysis, so we will drop it. 

In [50]:
cleaned_dataset=cleaned_dataset.drop(columns=['Species '])

'investigator or source' column has no use for statistical purposes but still has information that does have ofr individual cases

In [51]:
cleaned_dataset=cleaned_dataset.reset_index(drop=True)

This, our dataset is ready to be exported.

In [52]:
cleaned_dataset.to_csv('output/Sharks_attacks_cleaned.csv',index=False)